# Data source

Data for download: https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

Resources used as reference:
1) Training the image classifier to recognize different species of flowers:
https://www.kaggle.com/dtosidis/flower-classifier-tensorflow
  
2) Loading and preprocessing an image dataset
https://www.tensorflow.org/tutorials/load_data/images

3) Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

4) Image classification
https://www.tensorflow.org/tutorials/images/classification


In [38]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf

import PIL
import PIL.Image

import pathlib

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)

In [39]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


### Total images in the dataset

In [40]:
data_dir = 'flower_photos'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

## Image Classification
https://www.tensorflow.org/tutorials/images/classification?hl=zh-tw

In [41]:

batch_size = 32
img_height = 180
img_width = 180

## Generating datasets
https://keras.io/examples/vision/image_classification_from_scratch/

### Generating a training dataset

In [42]:
# When the subset below is defined as "training" the 0.2 validation split takes 80% of the data as the training set

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.05,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 3487 files for training.


### Generating a validation dataset

In [43]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the validation set

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.05,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 183 files for validation.


### Generating a test dataset

In [44]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the test set

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Class names

In [45]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


### Rescaling the data
https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling


In [46]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Normalizing the data (trainign and validation datasets)

In [47]:
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds =  val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


### Autotune is done to cache data and make processing and resource mgmt more effieicient

In [48]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [49]:
normalized_train_ds = normalized_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalized_val_ds = normalized_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [50]:
# num_classes = 5
num_classes = len(class_names)
num_classes

5

## Model 1 (Sequential Model)
https://www.tensorflow.org/guide/keras/sequential_model

In [51]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [15]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [16]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=16
)

Epoch 1/16
109/109 [==============================] - 25s 230ms/step - loss: 3137.6106 - accuracy: 0.3324 - val_loss: 1531.6011 - val_accuracy: 0.3443
Epoch 2/16
109/109 [==============================] - 7s 65ms/step - loss: 910.4540 - accuracy: 0.4118 - val_loss: 782.0704 - val_accuracy: 0.3607
Epoch 3/16
109/109 [==============================] - 7s 64ms/step - loss: 837.4989 - accuracy: 0.4310 - val_loss: 780.0682 - val_accuracy: 0.3661
Epoch 4/16
109/109 [==============================] - 7s 64ms/step - loss: 650.1601 - accuracy: 0.4677 - val_loss: 918.6453 - val_accuracy: 0.3880
Epoch 5/16
109/109 [==============================] - 7s 63ms/step - loss: 583.7142 - accuracy: 0.4792 - val_loss: 530.8185 - val_accuracy: 0.3880
Epoch 6/16
109/109 [==============================] - 7s 63ms/step - loss: 334.7931 - accuracy: 0.5053 - val_loss: 605.9751 - val_accuracy: 0.3224
Epoch 7/16
109/109 [==============================] - 7s 65ms/step - loss: 384.2635 - accuracy: 0.4677 - val_loss:

## Model 1 (Sequential Model) Epoch sensitivity

In [17]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [18]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [19]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=6
)

Epoch 1/6
109/109 [==============================] - 5s 44ms/step - loss: 1507.9851 - accuracy: 0.2108 - val_loss: 1.6087 - val_accuracy: 0.1913
Epoch 2/6
109/109 [==============================] - 5s 43ms/step - loss: 1.6145 - accuracy: 0.2314 - val_loss: 1.6067 - val_accuracy: 0.2404
Epoch 3/6
109/109 [==============================] - 5s 44ms/step - loss: 1.6101 - accuracy: 0.2461 - val_loss: 1.6051 - val_accuracy: 0.2404
Epoch 4/6
109/109 [==============================] - 5s 44ms/step - loss: 1.6057 - accuracy: 0.2455 - val_loss: 1.6039 - val_accuracy: 0.2404
Epoch 5/6
109/109 [==============================] - 5s 43ms/step - loss: 1.6021 - accuracy: 0.2452 - val_loss: 1.6032 - val_accuracy: 0.2404
Epoch 6/6
109/109 [==============================] - 5s 44ms/step - loss: 1.6012 - accuracy: 0.2452 - val_loss: 1.6027 - val_accuracy: 0.2404


## Model 1a (Sequential Model) With Normalized data

In [20]:
model_1a = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [21]:
model_1a.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [22]:
model_1a.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=6
)

Epoch 1/6
109/109 [==============================] - 11s 97ms/step - loss: 7.0116 - accuracy: 0.2114 - val_loss: 1.6089 - val_accuracy: 0.2404
Epoch 2/6
109/109 [==============================] - 5s 43ms/step - loss: 1.6065 - accuracy: 0.2532 - val_loss: 1.6051 - val_accuracy: 0.2404
Epoch 3/6
109/109 [==============================] - 5s 43ms/step - loss: 1.5882 - accuracy: 0.2656 - val_loss: 1.6695 - val_accuracy: 0.2514
Epoch 4/6
109/109 [==============================] - 5s 44ms/step - loss: 1.5634 - accuracy: 0.2756 - val_loss: 1.6035 - val_accuracy: 0.2404
Epoch 5/6
109/109 [==============================] - 5s 44ms/step - loss: 1.5996 - accuracy: 0.2466 - val_loss: 1.6029 - val_accuracy: 0.2404
Epoch 6/6
109/109 [==============================] - 5s 44ms/step - loss: 1.5959 - accuracy: 0.2492 - val_loss: 1.5995 - val_accuracy: 0.2459


## Model 2 

In [55]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [56]:
model_2.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [54]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=6
)

Epoch 1/6
 19/109 [====>.........................] - ETA: 56s - loss: 1.6414 - accuracy: 0.2039

KeyboardInterrupt: 

In [26]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=8
)

Epoch 1/8
109/109 [==============================] - 53s 486ms/step - loss: 0.2391 - accuracy: 0.9191 - val_loss: 1.1211 - val_accuracy: 0.6776
Epoch 2/8
109/109 [==============================] - 53s 485ms/step - loss: 0.1985 - accuracy: 0.9297 - val_loss: 1.4030 - val_accuracy: 0.6230
Epoch 3/8
109/109 [==============================] - 54s 496ms/step - loss: 0.1577 - accuracy: 0.9484 - val_loss: 1.5751 - val_accuracy: 0.5956
Epoch 4/8
109/109 [==============================] - 58s 534ms/step - loss: 0.1120 - accuracy: 0.9667 - val_loss: 1.5745 - val_accuracy: 0.6557
Epoch 5/8
109/109 [==============================] - 55s 502ms/step - loss: 0.0855 - accuracy: 0.9751 - val_loss: 1.7683 - val_accuracy: 0.6667
Epoch 6/8
109/109 [==============================] - 56s 512ms/step - loss: 0.0599 - accuracy: 0.9828 - val_loss: 1.8698 - val_accuracy: 0.6612
Epoch 7/8
109/109 [==============================] - 53s 490ms/step - loss: 0.0277 - accuracy: 0.9937 - val_loss: 2.3742 - val_accuracy:

In [27]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
109/109 [==============================] - 55s 505ms/step - loss: 0.0123 - accuracy: 0.9983 - val_loss: 2.0296 - val_accuracy: 0.6831
Epoch 2/10
109/109 [==============================] - 56s 512ms/step - loss: 0.0079 - accuracy: 0.9989 - val_loss: 2.0692 - val_accuracy: 0.6885
Epoch 3/10
109/109 [==============================] - 54s 497ms/step - loss: 0.0055 - accuracy: 0.9994 - val_loss: 2.0980 - val_accuracy: 0.6995
Epoch 4/10
109/109 [==============================] - 54s 492ms/step - loss: 0.0047 - accuracy: 0.9994 - val_loss: 2.1237 - val_accuracy: 0.6940
Epoch 5/10
109/109 [==============================] - 54s 491ms/step - loss: 0.0043 - accuracy: 0.9994 - val_loss: 2.1358 - val_accuracy: 0.7049
Epoch 6/10
109/109 [==============================] - 53s 491ms/step - loss: 0.0039 - accuracy: 0.9994 - val_loss: 2.1403 - val_accuracy: 0.7049
Epoch 7/10
109/109 [==============================] - 54s 500ms/step - loss: 0.0037 - accuracy: 0.9994 - val_loss: 2.1474 - val_ac

In [28]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=12
)

Epoch 1/12
109/109 [==============================] - 61s 557ms/step - loss: 0.0029 - accuracy: 0.9994 - val_loss: 2.1815 - val_accuracy: 0.6940
Epoch 2/12
109/109 [==============================] - 60s 548ms/step - loss: 0.0041 - accuracy: 0.9994 - val_loss: 2.2163 - val_accuracy: 0.6885
Epoch 3/12
109/109 [==============================] - 64s 590ms/step - loss: 0.0033 - accuracy: 0.9994 - val_loss: 2.1172 - val_accuracy: 0.7213
Epoch 4/12
109/109 [==============================] - 62s 565ms/step - loss: 0.0029 - accuracy: 0.9994 - val_loss: 2.1750 - val_accuracy: 0.6940
Epoch 5/12
109/109 [==============================] - 58s 533ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 2.1796 - val_accuracy: 0.6885
Epoch 6/12
109/109 [==============================] - 58s 529ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 2.1707 - val_accuracy: 0.6885
Epoch 7/12
109/109 [==============================] - 65s 593ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 2.1801 - val_ac

In [29]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=14
)

Epoch 1/14
109/109 [==============================] - 62s 564ms/step - loss: 0.0015 - accuracy: 0.9994 - val_loss: 2.1759 - val_accuracy: 0.7049
Epoch 2/14
109/109 [==============================] - 65s 595ms/step - loss: 0.0209 - accuracy: 0.9948 - val_loss: 2.6731 - val_accuracy: 0.6175
Epoch 3/14
109/109 [==============================] - 65s 593ms/step - loss: 0.2473 - accuracy: 0.9252 - val_loss: 1.7773 - val_accuracy: 0.6831
Epoch 4/14
109/109 [==============================] - 69s 633ms/step - loss: 0.0882 - accuracy: 0.9736 - val_loss: 2.2340 - val_accuracy: 0.6393
Epoch 5/14
109/109 [==============================] - 63s 575ms/step - loss: 0.0423 - accuracy: 0.9868 - val_loss: 3.1447 - val_accuracy: 0.5847
Epoch 6/14
109/109 [==============================] - 58s 531ms/step - loss: 0.0532 - accuracy: 0.9837 - val_loss: 2.4214 - val_accuracy: 0.6175
Epoch 7/14
109/109 [==============================] - 58s 530ms/step - loss: 0.0368 - accuracy: 0.9914 - val_loss: 2.3854 - val_ac

In [31]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=16
)

Epoch 1/16
109/109 [==============================] - 45s 411ms/step - loss: 0.0016 - accuracy: 0.9994 - val_loss: 2.2160 - val_accuracy: 0.7049
Epoch 2/16
109/109 [==============================] - 46s 421ms/step - loss: 0.0014 - accuracy: 0.9994 - val_loss: 2.2378 - val_accuracy: 0.6995
Epoch 3/16
109/109 [==============================] - 50s 461ms/step - loss: 0.0012 - accuracy: 0.9994 - val_loss: 2.2653 - val_accuracy: 0.7104
Epoch 4/16
109/109 [==============================] - 51s 467ms/step - loss: 0.0016 - accuracy: 0.9994 - val_loss: 2.2614 - val_accuracy: 0.7049
Epoch 5/16
109/109 [==============================] - 54s 494ms/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 2.2810 - val_accuracy: 0.6940
Epoch 6/16
109/109 [==============================] - 50s 462ms/step - loss: 0.0013 - accuracy: 0.9994 - val_loss: 2.2835 - val_accuracy: 0.6940
Epoch 7/16
109/109 [==============================] - 52s 477ms/step - loss: 0.0012 - accuracy: 0.9994 - val_loss: 2.2977 - val_ac

In [32]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=18
)

Epoch 1/18
109/109 [==============================] - 54s 494ms/step - loss: 8.7196e-04 - accuracy: 0.9994 - val_loss: 2.3502 - val_accuracy: 0.7158
Epoch 2/18
109/109 [==============================] - 57s 524ms/step - loss: 7.9307e-04 - accuracy: 0.9994 - val_loss: 2.3476 - val_accuracy: 0.7104
Epoch 3/18
109/109 [==============================] - 55s 508ms/step - loss: 7.3871e-04 - accuracy: 0.9994 - val_loss: 2.3512 - val_accuracy: 0.7104
Epoch 4/18
109/109 [==============================] - 58s 532ms/step - loss: 7.0824e-04 - accuracy: 0.9994 - val_loss: 2.3576 - val_accuracy: 0.7104
Epoch 5/18
109/109 [==============================] - 58s 529ms/step - loss: 6.8425e-04 - accuracy: 0.9994 - val_loss: 2.3667 - val_accuracy: 0.7104
Epoch 6/18
109/109 [==============================] - 56s 513ms/step - loss: 6.7293e-04 - accuracy: 0.9994 - val_loss: 2.3699 - val_accuracy: 0.7158
Epoch 7/18
109/109 [==============================] - 55s 501ms/step - loss: 6.4788e-04 - accuracy: 0.9994

In [57]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

109/109 [==============================] - 88s 810ms/step - loss: 1.2256 - accuracy: 0.4832 - val_loss: 1.0711 - val_accuracy: 0.6011


In [58]:
model_2.save('model_2')

NotFoundError: Failed to create a NewWriteableFile: model_2\variables\variables_temp_d8965615a90945f2be889de340f7d15d/part-00000-of-00001.data-00000-of-00001.tempstate18400128457331505972 : The system cannot find the path specified.
; No such process [Op:SaveV2]